In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [23]:
# input
pvar_f   <- '/oak/stanford/groups/mrivas/ukbb24983/cnv/pgen/cnv.pvar'
csq_rules_f <- '../VEP_consequence_group.tsv'
vep_f    <- '/scratch/groups/mrivas/ukbb24983/cnv/annotation_20201003/cnv.vep101-loftee.20201009.tsv.gz'
# output
vep_csq_f <- str_replace(vep_f, '.tsv.gz$', '.Csq.tsv')


In [3]:
csq_rules_f %>% fread(select=c('#Consequence', 'Csq')) %>%
rename('Consequence'='#Consequence') -> csq_rules_df


In [4]:
vep_f %>% fread(colClasses = c('#CHROM'='character')) %>%
rename('CHROM'='#CHROM') -> vep_df


In [16]:
pvar_f %>% fread(colClasses = c('#CHROM'='character')) %>% rename('CHROM'='#CHROM') %>%
separate(ID, c(NA, 'POS_s', 'POS_e', NA), remove=F) %>%
mutate(pvar_order = 1:n()) -> pvar_df


In [25]:
pvar_df %>%
left_join(vep_df %>% select(-CHROM, -POS), by='ID') %>%
left_join(csq_rules_df, by='Consequence') %>%
arrange(as.numeric(CHROM), as.numeric(POS_s), as.numeric(POS_e), pvar_order) %>%
select(-FILTER) -> df


In [26]:
df %>%
rename('#CHROM' = 'CHROM') %>%
fwrite(vep_csq_f, sep='\t', na = "NA", quote=F)


## check the `Csq` assignments

In [21]:
df %>% count(Csq)

Csq,n
<chr>,<int>
intron,54188
others,64440
pav,1143
pcv,774
ptv,152268
utr,2117
NA,250


We do not have variant annotation for 250 variants.


In [22]:
vep_df %>%
count(Consequence) %>%
left_join(csq_rules_df, by='Consequence') %>%
arrange(Csq, -n)

Consequence,n,Csq
<chr>,<int>,<chr>
intron_variant,54188,intron
intergenic_variant,24390,others
TFBS_ablation,16525,others
upstream_gene_variant,7583,others
downstream_gene_variant,5395,others
regulatory_region_ablation,3974,others
non_coding_transcript_exon_variant,3458,others
regulatory_region_variant,2880,others
TF_binding_site_variant,222,others
